In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
import os
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    def extract_last_bracketed(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None
    df[write_col] = df[read_col].apply(extract_last_bracketed)
    return df

def validate_text_column(df, df_reference):
    reference_texts = set(df_reference['text'].unique())
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, 'output', 'extracted_label')
    return df

for i in [1]:
    print('\n\nMUMER', i, '\n\n')
    
    model_tag = 'llama3'
    dataset_tag = 'sst5'
    prompt = 'cot_random_samples_cohere'
    temp = 0.3
    number_of_experimet = i
    selected_samples = f'{model_tag}_random42_60'
    
    model = get_model_by_tag(model_tag, token, temp)
    prompt_txt = get_prompt_template(dataset_tag, prompt)
    examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

    output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    try:
        df_old = pd.read_csv(output_path)
    except FileNotFoundError:
        df_old = pd.DataFrame(columns=['text', 'label', 'output'])

    # Dodaj 'original_index', żeby móc później przywrócić kolejność
    full_dataset = get_dataset(dataset_tag=dataset_tag)
    full_dataset = full_dataset.reset_index().rename(columns={'index': 'original_index'})

    # Wczytaj stare wyniki lub stwórz pusty DataFrame
    try:
        df_old = pd.read_csv(output_path)
    except FileNotFoundError:
        df_old = pd.DataFrame(columns=['text', 'label', 'output', 'error'])
        
    # Do zaannotowania:
    # - brak tekstu w df_old
    # - lub w df_old jest 'error' niepuste dla danego textu
    if 'error' not in df_old.columns:
        df_old['error'] = None
    texts_with_errors = df_old[df_old['error'].notna() & (df_old['error'].astype(str) != '')]['text']
    texts_already_done = set(df_old['text']) - set(texts_with_errors)
    df_to_annotate = full_dataset[~full_dataset['text'].isin(texts_already_done)]

    print('Do zaannotowania:', len(df_to_annotate))


    if not df_to_annotate.empty:
        annotator = LLMAnnotator(
            model=model,
            dataset=df_to_annotate,
            examples_for_prompt=examples_for_prompt,
            prompt_template=prompt_txt,
            column_text="text",
            column_label="label",
            column_output="output"
        )
        df_new = annotator.get_results()
        df_new = validate_text_column(pd.DataFrame(df_new), examples_for_prompt)

        # Łączenie z zachowaniem indeksu
        
        df_new = df_new.merge(df_to_annotate[['text', 'original_index']], on='text', how='left')

        df_old = df_old.merge(full_dataset[['text', 'original_index']], on='text', how='left')
        
        df_combined = pd.concat([df_old, df_new], ignore_index=True)
        df_combined.drop_duplicates(subset='original_index', keep='last', inplace=True)
        df_combined.sort_values(by='original_index', inplace=True)
        
        # Usuń kolumnę 'error', jeśli wszystkie wartości są puste lub puste stringi
        if 'error' in df_combined.columns and df_combined['error'].replace('', pd.NA).isna().all():
            df_combined.drop(columns='error', inplace=True)

        df_combined.to_csv(output_path, index=False)
        print("Zaktualizowano plik:", output_path)

    else:
        print("Brak nowych przykładów do anotacji.")




MUMER 1 


Do zaannotowania: 500


/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: To analyze the given text and determine the most suitable rating, let's break down the process step by step.

**Step 1: Summarize the main topic of the text.**

The main topic of the text appears to be a review or critique of a film, specifically Chin's film. The text mentions the concept of an "e-graveyard," which suggests that the film might be exploring themes related to technology, innovation, or failed ideas.

**Step 2: Identify any key terms or phrases that hint at a specific rating.**

The key phrases that stand out in the text are:

* "holds as many good ideas as bad" - This phrase suggests that the film has a mix of both positive and negative aspects, implying a somewhat neutral or balanced evaluation.
* "cold comfort" - This phrase has a slightly negative connotation, implying that the film's message or themes might be somewhat disappointing or unfulfilling.
* "serves up with style and empathy" - This phrase has a positive connotation, suggesting that t

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. The content length of 212116747 bytes exceeds the maximum size limit of 20971520 bytes. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by ReadTimeoutError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Read timed out. (read timeout=90.001)"))'))
Content-Length: 212116747
API Key: lsv2_********************************************7etrace=877ded4e-ae76-441c-ba9b-70255d2ec3a0,id=877ded4e-ae76-441c-ba9b-70255d2ec3a0


In [5]:
output_path

'./results/cot_random_samples_cohere_temp0.3/llama3/sst5_llama3_cot_random_samples_cohere_llama3_random42_60_temp0.3_exp1.csv'

In [6]:
df_combined

,text,output,logprobs,top_logprobs,original_label,original_index,was_in_selected_samples,extracted_label
0,Though the opera itself takes place mostly ind...,Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",3,0,NaN,NaN
1,If you can read the subtitles ( the opera is s...,Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",4,1,NaN,NaN
2,( `` Safe Conduct '' ) is a long movie at 163 ...,Let's analyze the text step by step:\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",4,2,NaN,NaN
3,The beauty of Alexander Payne 's ode to the Ev...,Let's break down the analysis step by step:\n\...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",4,3,NaN,NaN
4,The fourth in a series that I 'll bet most par...,To determine the most appropriate rating for t...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",3,4,NaN,NaN
...,...,...,...,...,...,...,...,...
995,"In other words , it 's badder than bad .",To analyze the given text and determine the mo...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",1,995,False,1
996,The direction occasionally rises to the level ...,Let's analyze the text step by step:\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",2,996,False,1
997,The movie is ingenious fun .,To determine the most appropriate rating for t...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",5,997,False,4
998,The cast comes through even when the movie doe...,To determine the most appropriate rating for t...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",3,998,False,3
